<a href="https://colab.research.google.com/github/pepperamy/tenK_phase2/blob/main/10k_featureselection_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pickle
import re
from matplotlib import pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.optimize import linear_sum_assignment
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import mstats
import math

from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, precision_recall_curve, classification_report,accuracy_score, auc, roc_curve, roc_auc_score, average_precision_score

In [ ]:
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from keras.layers import Embedding, Dense, Conv1D, MaxPooling1D, RNN,\
Dropout, Activation, Input, Flatten, Concatenate, BatchNormalization, Lambda, LSTM
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras import regularizers
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import models
from keras import layers

In [ ]:
from sklearn.utils import class_weight

In [ ]:
import random

In [ ]:
df = pd.read_csv('data_performance_words_win1_03.csv')

In [ ]:
df.head()

,cik,y1,y2,s1,window,label,sic_class_x,rsst_acc,ch_rec,ch_inv,soft_asset,ch_cs,ch_roa,issue,aqi,asset_turnover,cfed,depi,gmi,ig,opm,rg,sg,sgee,pastavg5,pastavg3,pastavg1,cr5,cr3,cr1,WeakModal_3_avg,WeakModal_3_dis,WeakModal_3_n,WeakModal_3_new,WeakModal_3_p,WeakModal_3_u,Litigious_3_avg,Litigious_3_dis,Litigious_3_n,Litigious_3_new,...,Differ_3_n,Differ_3_new,Differ_3_p,Differ_3_u,Drives_3_avg,Drives_3_dis,Drives_3_n,Drives_3_new,Drives_3_p,Drives_3_u,Affiliation_3_avg,Affiliation_3_dis,Affiliation_3_n,Affiliation_3_new,Affiliation_3_p,Affiliation_3_u,Achieve_3_avg,Achieve_3_dis,Achieve_3_n,Achieve_3_new,Achieve_3_p,Achieve_3_u,Power_3_avg,Power_3_dis,Power_3_n,Power_3_new,Power_3_p,Power_3_u,Reward_3_avg,Reward_3_dis,Reward_3_n,Reward_3_new,Reward_3_p,Reward_3_u,Risk_3_avg,Risk_3_dis,Risk_3_n,Risk_3_new,Risk_3_p,Risk_3_u
0,20.0,1995.0,1996.0,0.633505,1.0,0.0,3,0.063117,-0.072425,-0.067847,0.661974,-0.147079,0.198730,1,0.974056,1.624273,0.206066,1.226305,0.914198,0.715334,0.044798,0.783539,0.814093,1.013528,NaN,NaN,NaN,NaN,NaN,NaN,-0.000071,0.000000,0.095238,0.004950,0.047619,0.761905,-0.002775,0.002801,0.190476,0.000000,...,0.380952,0.004950,0.380952,0.142857,-0.007085,0.051165,0.428571,0.054325,0.285714,0.190476,0.002484,0.017475,0.095238,0.004950,0.285714,0.523810,-0.003165,0.017510,0.285714,0.013158,0.047619,0.571429,-0.004527,0.016180,0.428571,0.029638,0.285714,0.190476,-0.000009,0.002644,0.190476,0.000000,0.190476,0.523810,-0.002433,0.002488,0.333333,0.019737,0.000000,0.571429
1,20.0,1996.0,1997.0,0.803827,1.0,0.0,3,0.066126,-0.018252,-0.059281,0.620435,-0.066028,0.034753,1,1.248039,1.606518,0.025826,1.088859,0.978102,0.755016,0.062466,0.938786,0.969746,1.012432,0.633505,0.633505,0.633505,1.268856,1.268856,1.268856,-0.000034,0.000000,0.066667,0.028571,0.000000,0.866667,0.003234,0.000000,0.133333,0.000000,...,0.266667,0.028571,0.133333,0.533333,-0.004321,0.044777,0.266667,0.000000,0.266667,0.400000,-0.000118,0.004586,0.200000,0.000000,0.266667,0.466667,0.001261,0.008828,0.066667,0.000000,0.133333,0.733333,-0.003780,0.018289,0.333333,0.000000,0.200000,0.400000,0.000668,0.007947,0.200000,0.000000,0.200000,0.533333,-0.001054,0.013954,0.066667,0.000000,0.066667,0.800000
2,20.0,1997.0,1998.0,0.462705,1.0,0.0,3,0.095900,0.066711,0.013223,0.656729,-0.030720,0.019574,1,0.840540,1.574474,0.020191,0.991780,0.991078,1.073227,0.073961,1.241132,1.022834,0.983269,0.718666,0.718666,0.803827,0.643838,0.643838,0.575627,0.002823,0.005714,0.066667,0.007841,0.133333,0.466667,-0.001499,0.004000,0.133333,0.004315,...,0.400000,0.032995,0.200000,0.066667,-0.003528,0.048389,0.266667,0.038724,0.266667,0.133333,-0.001334,0.007739,0.266667,0.003683,0.200000,0.200000,-0.002187,0.011111,0.266667,0.008254,0.066667,0.333333,0.000062,0.028035,0.266667,0.021889,0.266667,0.133333,-0.003310,0.001504,0.266667,0.002960,0.000000,0.400000,-0.000069,0.011111,0.066667,0.004688,0.066667,0.533333
3,20.0,1998.0,1999.0,0.701689,1.0,0.0,3,0.054997,0.026323,-0.009875,0.676757,0.011434,0.002424,1,0.837833,1.604656,0.003715,1.025961,1.024634,0.948804,0.076906,1.077020,0.985921,0.983057,0.633346,0.633346,0.462705,1.107909,1.107909,1.516495,0.000590,0.006755,0.045455,0.000000,0.136364,0.545455,-0.000112,0.005460,0.090909,0.000000,...,0.136364,0.011462,0.136364,0.454545,0.001771,0.051960,0.045455,0.051891,0.227273,0.454545,0.000279,0.003730,0.090909,0.014010,0.045455,0.590909,-0.000451,0.010119,0.045455,0.010288,0.090909,0.590909,0.001965,0.034894,0.090909,0.019415,0.136364,0.500000,0.000347,0.003575,0.000000,0.008179,0.045455,0.681818,-0.000126,0.003620,0.090909,0.000000,0.090909,0.545455
4,20.0,1999.0,2000.0,0.823023,1.0,0.0,3,-0.024892,-0.024929,0.041267,0.705573,-0.001713,-0.014429,1,1.093648,1.560280,-0.015406,0.942174,0.995699,1.221034,0.068754,0.933610,0.966150,1.000184,0.650432,0.656074,0.701689,1.265350,1.254468,1.172917,-0.000791,0.002083,0.176471,0.000000,0.000000,0.764706,-0.000278,0.003717,0.117647,0.000000,...,0.352941,0.000000,0.176471,0.411765,0.030122,0.041

In [ ]:
df_fl = df[(df.y2 <= 2012) & (df.y2 >= 1995 ) ]

In [ ]:
selected_new = ['WeakModal', 'Litigious', 'StrongModal','Negative', 'Positive', 'Uncertainty', 
        'Compare', 'Achieve', 'Discrep', 'Reward']

v_perf = ['aqi',
 'asset_turnover',
 'depi',
 'gmi',
 #'ig',
 'opm',
 'rg',
 'sg',
 'sgee',
 'ch_rec',
 'ch_inv',
 'soft_asset',
 'ch_cs',
 'ch_roa',
 'issue']

v_1 = ['s1']
v_2 = ['pastavg3','cr3']

In [ ]:
selected_new_all = []
temp = []
for s in selected_new:
    wrd = s.split('_')[0]
    if wrd not in temp:
        #print(s,'\n',temp)
        #selected_new_all.append(wrd+'_3_avg')
        selected_new_all.append(wrd+'_3_p')
        selected_new_all.append(wrd+'_3_n')
        #selected_new_all.append(wrd+'_3_u')
        selected_new_all.append(wrd+'_3_new')
        selected_new_all.append(wrd+'_3_dis')
        temp.append(wrd)
    else: 
        pass

In [ ]:
from scipy import stats

In [ ]:
def ttest(df, name, feature):
    fraud1 = df[df.label==1][feature].dropna()
    nonfraud0 = df[df.label==0][feature].dropna()

    # print('mean1: ',np.mean(fraud1),' mean0: ',np.mean(nonfraud0))
    # print('std1: ',np.std(fraud1),' std0: ',np.std(nonfraud0))
    # fig, ax = plt.subplots() 
    # fig.set_figheight(3)
    # fig.set_figwidth(3)
    # ax.boxplot([fraud1,nonfraud0],showfliers=False,widths = 0.4,showmeans=True)
    # ax.set_xticklabels([1,0])
    # ax.set_title(name)
    # #ax.set_aspect(2)
    # plt.show()
    return stats.ttest_ind(fraud1,nonfraud0)

In [ ]:
ttest_res = {}
for s in selected_new_all:
  t_res = ttest(df_fl, '', s)
  ttest_res[s] = [t_res[0],t_res[1]]

In [ ]:
ttest_res_tb = pd.DataFrame(ttest_res,index = ['t_value', 'p-value'])

In [ ]:
ttest_res_tb = ttest_res_tb.T
ttest_res_tb.to_csv('ttest_res20200301.csv')

In [ ]:
df_fl_1 = df_fl.copy()

In [ ]:
selected_comb = []
for s in selected_new: 
  df_fl_1[s+'_up'] = df_fl_1[s + '_3_p'] + df_fl_1[s + '_3_new']
  selected_comb.append(s+'_up')
  df_fl_1[s+'_down'] = df_fl_1[s + '_3_n'] + df_fl_1[s + '_3_dis']
  selected_comb.append(s+'_down')
  

In [ ]:
df_fl_1.shape


(53635, 212)

In [ ]:
df_fl_1.to_csv('data_performance_words_win1_comb_20210301.csv',index = False)

In [ ]:
ttest_res_comb = {}
for s in selected_comb:
  t_res = ttest(df_fl_1, '', s)
  ttest_res_comb[s] = [t_res[0],t_res[1]]
ttest_res_comb_tb = pd.DataFrame(ttest_res_comb, index = ['t_value', 'p-value'])
ttest_res_comb_tb.to_csv('ttest_res_comb20200301.csv')

In [94]:
def model_M1(n1,n2):
    model_m1 = Sequential(name = 'M1')
    model_m1.add(BatchNormalization())
    model_m1.add(layers.Dense(n1,name = 'layer_1',activation='relu',kernel_initializer='he_uniform')) 
    model_m1.add(layers.Dropout(0.3))
    #model_m1.add(layers.Dense(64,name = 'layer_2'))
    model_m1.add(layers.Dense(n2,name = 'layer_2',activation='relu'))
    model_m1.add(layers.Dense(1,activation='sigmoid'))
    #model_m1.summary()
    return model_m1

In [95]:
def performance_measure(pred_yp, y):
    '''
    Given lists of predicted y probability and x, y, return a dataframe of AR, AUC, Brier, Decile Table
    '''
    
    tenc_dat = pd.DataFrame({'y_true':y,'probability':pred_yp.flatten()})
    tenc_dat.sort_values('probability',axis = 0,ascending=False, inplace = True)
    tenc_dat.index = range(0,len(tenc_dat))
    y = tenc_dat['y_true']
    point = float(len(tenc_dat))/10
    point = int(round(point))
    tenc = []
    for i in range(0,10):
        tenc.append(y[(i*point):((i+1)*point)])
    tenc[9]=tenc[9].append(y[10*point:])
    total = sum(y)
    num_of_bkr = []
    for j in range(0,10):
        num_of_bkr.append(sum(tenc[j]))
    tencile_bkr = np.array(num_of_bkr)
    rate = tencile_bkr.astype(float)/total

    return rate

In [96]:
class AUCEvaluation(Callback):
    """ Show AUC after interval number of epoches """
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()
        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            logs['auc'] = score
            tencile=performance_measure(y_pred, self.y_val)
            logs['tencile'] = tencile
            print(" epoch:{:d} auc: {:.4f}".format(epoch, score))



In [97]:
def fit_model( model ,x_train, y_train, x_val, y_val, x_test, y_test, model_name, verbose = 0):
    
    history = None
    result = None
    
    model_df = model
    
    class_weights = class_weight.compute_class_weight('balanced',
                              np.unique(y_train),
                              y_train)
    class_weights = {l:c for l,c in zip(np.unique(y_train), class_weights)}
    print(class_weights)
    # val_class_weights = class_weight.compute_class_weight('balanced',
    #                             np.unique(y_val),
    #                           y_val)
    #print(class_weights, val_class_weights)
    #class_weights = [0.50328272, 30]
    #val_class_weights = [ 0.50328272, 30]
#     print(class_weights)
#     print(val_class_weights)
    
    val_sample_weights = []
    for y in y_val:
        if y == 1:
          val_sample_weights.append(class_weights[1])
        else: val_sample_weights.append(class_weights[0])
    val_sample_weights = np.asarray(val_sample_weights)
    
    
    #opt = optimizers.RMSprop(lr=0.0005)
    model_df.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['acc'])
    
    auc_eval = AUCEvaluation(validation_data=(x_val, y_val), interval=1)
    #earlyStopping = EarlyStopping(monitor='auc',patience = 5, verbose =verbose, mode ='max')
    #checkpoint = ModelCheckpoint(model_name,monitor='auc',verbose=verbose,save_best_only=True, mode ='max')
    
    history = model_df.fit(x_train,
                y_train,
                epochs=25,
                batch_size=128,
                verbose =verbose,
                callbacks=[auc_eval],\
                #callbacks=[auc_eval, earlyStopping,checkpoint], #checkpoint
                class_weight = class_weights, \
                validation_data=(x_val,y_val,val_sample_weights)
                          ) #
    #model_df.load_weights(model_name)
    y_pred = model_df.predict(x_test)
    y_pred_1=np.reshape(y_pred, -1)
    auc_score = roc_auc_score(y_test, y_pred_1)
    #auc_score = roc_auc_score(y_test, y_pred_1)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_1)
    print("AUC: " ,round(auc_score,4))
        
    
    #y_pred_2= [1 if y >0.1 else 0 for y in np.reshape(y_pred_1, -1)] 
    #print(classification_report(y_test, y_pred_2))
    #print(confusion_matrix(y_test, y_pred_2))
    #print("AUC: " ,round(auc_score,4))
    
#     precision, recall, _ = precision_recall_curve(y_test,  y_pred_1)
#     plt.plot(recall, precision, marker='.', label='Precision-Recall')
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.legend(loc=4)
#     plt.show()                                                  
    
#     fpr, tpr, _ = roc_curve(y_test,  y_pred_1)
#     plt.plot(fpr,tpr,label="auc="+str(round(auc_score,4)))
#     plt.legend(loc=4)
#     plt.show()
        
    return history, auc_score, fpr, tpr, thresholds, [y_test, y_pred_1]


In [98]:
def cross_val(n1, n2, data,label,columns_fl,model_name):
    tprs = []
    auc_list = []
    mean_fpr = np.linspace(0,1,100)
    
    kf = StratifiedKFold(n_splits = 3, shuffle = True)
    c = 0
    data_1 = data[columns_fl + [label]]
       
    print("# of obs: {}".format(len(data_1)))
    
    data_1 = data_1.dropna()
    data_target = data_1.reset_index(drop = True)
    
    print("# of obs after drop na: {}".format(len(data_target)))
    print("# of positive obs: {}".format(data_target[label].sum()))
    
    predicted_res =[]
    
    for train_index, test_index in kf.split(data_target[columns_fl],data_target[label]):
        
        c += 1
        
        n1 = len(columns_fl)*2
        n2 = len(columns_fl)
        model = model_M1(n1,n2)
    

        train_data = data_target.loc[train_index,columns_fl].to_numpy()
        train_label = data_target.loc[train_index,label]
        test_x = data_target.loc[test_index,columns_fl].to_numpy()
        test_y = data_target.loc[test_index,label] 

        print('iterate_num: ', c, '\n','sum of test_y:', sum(test_y))
        # train_x, val_x, train_y, val_y = train_test_split(train_data.to_numpy(), train_label.to_numpy(),test_size=0.2,                                                             random_state=42, stratify = train_label)
        # print('train_x_shape',train_x.shape)
        mod_res = fit_model(model, train_data, train_label, test_x, test_y, test_x, test_y, model_name)
        auc_list.append(mod_res[1])
        tprs.append(np.interp(mean_fpr, mod_res[2], mod_res[3]))
#         print(type(mod_res[-1]))
#         print(len(mod_res[-1]))
#         print(mod_res[-1])
        temp_pred_res = [mod_res[-1],test_index]
        #print(temp_pred_res)
        predicted_res.append(temp_pred_res)
        
    mean_tpr = np.mean(tprs, axis=0)
    print('avg_AUC : ', np.mean(auc_list))
    print('avg_AUC_2 : ', auc(mean_fpr, mean_tpr))
    
    return np.mean(auc_list), mean_tpr, predicted_res

In [99]:
selected_comb_1 = [s for s in selected_comb if s not in ['Litigious_down', 'StrongModal_up', 'Negative_down', 'Positive_up', 'Reward_up', 'Reward_down']]

In [100]:
print("\n ======= Select columns for NN ALL OBS.=========")

result = []

# First choose all varaibles
cols = v_1 + v_2 + selected_new_all + v_perf
dataset = df_fl_1
#cross_val(data, label, columns_fl, name,w) 
n1 = 2*len(cols)
n2 = len(cols)
label = 'label'
h = cross_val( n1, n2, dataset, label, cols, 'NN')
    
best_auc = h[0]




 ======= Select columns for NN ALL OBS.=========
# of obs: 53635
# of obs after drop na: 45611
# of positive obs: 407.0
iterate_num:  1 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.6429
 epoch:1 auc: 0.6826
 epoch:2 auc: 0.6897
 epoch:3 auc: 0.6915
 epoch:4 auc: 0.6898
 epoch:5 auc: 0.6899
 epoch:6 auc: 0.6942
 epoch:7 auc: 0.7013
 epoch:8 auc: 0.6970
 epoch:9 auc: 0.6964
 epoch:10 auc: 0.7017
 epoch:11 auc: 0.6952
 epoch:12 auc: 0.7017
 epoch:13 auc: 0.6955
 epoch:14 auc: 0.6978
 epoch:15 auc: 0.7001
 epoch:16 auc: 0.6920
 epoch:17 auc: 0.6952
 epoch:18 auc: 0.7014
 epoch:19 auc: 0.6993
 epoch:20 auc: 0.6951
 epoch:21 auc: 0.6888
 epoch:22 auc: 0.6898
 epoch:23 auc: 0.6984
 epoch:24 auc: 0.6894
AUC:  0.6894
iterate_num:  2 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.6268
 epoch:1 auc: 0.6691
 epoch:2 auc: 0.6881
 epoch:3 auc: 0.6919
 epoch:4 auc: 0.7037
 epoch:5 auc: 0.7071
 epoch:6 auc: 0.7091

In [ ]:
gg# Next, remove one from the list each time

all_cols = selected_comb + v_1 + v_2
random.shuffle(all_cols)
#all_cols = cols

selected_cols = all_cols

for col in all_cols:
    
    print("\n ===========Test {}============\n".format(col))
    
    test_cols = [item for item in selected_cols if item !=col]
    
    n1 = 2*len(test_cols)
    n2 = len(test_cols)
    label = 'label'
    h = cross_val( n1, n2, df_fl_1, label,  v_perf +test_cols, 'NN')
    
    result.append([col, h[0], best_auc])
    
    if h[0]>= best_auc:  # better to drop col
        
        selected_cols = test_cols
        
        best_auc = h[0]
        
        print("*** {} removed ***".format(col))
    
    print("current auc: {0:.4f}\tbest_auc: {1:.4f}\tcolumn: {2}".format(h[0], best_auc, col))
    print("selected columns: ", selected_cols)


pd.DataFrame(result, columns = ["column", "auc", "prev_auc"]).to_csv("var_selection_comb_nn.csv")    
    
print("\n ======= DONE =========")

In [ ]:
print("\n ======= Select columns for NN one by one.=========")

result = []

# First choose all varaibles
cols = v_perf + v_1+ v_2 #+ selected_new_all

#cross_val(data, label, columns_fl, name,w) 
n1 = 2*len(cols)
n2 = len(cols)
label = 'label'
h = cross_val( n1, n2, df_fl, label, cols, 'NN')
    
best_auc = h[0]

# Next, remove one from the list each time




 ======= Select columns for LSTM ALL OBS.=========
# of obs: 53635
# of obs after drop na: 45611
# of positive obs: 407.0
iterate_num:  1 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.5926
 epoch:1 auc: 0.6235
 epoch:2 auc: 0.6408
 epoch:3 auc: 0.6442
 epoch:4 auc: 0.6499
 epoch:5 auc: 0.6552
 epoch:6 auc: 0.6575
 epoch:7 auc: 0.6571
 epoch:8 auc: 0.6578
 epoch:9 auc: 0.6571
AUC:  0.6571
iterate_num:  2 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.6197
 epoch:1 auc: 0.6629
 epoch:2 auc: 0.6906
 epoch:3 auc: 0.6969
 epoch:4 auc: 0.6971
 epoch:5 auc: 0.7017
 epoch:6 auc: 0.6997
 epoch:7 auc: 0.7016
 epoch:8 auc: 0.7079
 epoch:9 auc: 0.7113
AUC:  0.7113
iterate_num:  3 
 sum of test_y: 135.0
{0.0: 0.5045128749668171, 1.0: 55.89705882352941}
 epoch:0 auc: 0.5670
 epoch:1 auc: 0.6060
 epoch:2 auc: 0.6244
 epoch:3 auc: 0.6348
 epoch:4 auc: 0.6360
 epoch:5 auc: 0.6409
 epoch:6 auc: 0.6450
 epoch:7 auc: 0

In [ ]:
random.shuffle(selected_new_all)
#all_cols = selected_new_all

# selected_cols = cols

for col in selected_new_all:
    
    print("\n ===========Test {}============\n".format(col))
    
    #test_cols = [item for item in selected_cols if item !=col]
    test_cols = v_perf + v_1+ v_2 + [col]
    
    n1 = 2*len(test_cols)
    n2 = len(test_cols)
    label = 'label'
    h = cross_val( n1, n2, df_fl, label, test_cols, 'NN')

    gap = h[0] - best_auc
    print('****** gap: {0:.4f}'.format(gap))
    
    result.append([col, h[0], best_auc, gap])
    
    # if h[0]>= best_auc:  # better to drop col
        
    #     selected_cols = test_cols
        
    #     best_auc = h[0]
        
    #     print("*** {} removed ***".format(col))
    
    print("current auc: {0:.4f}\tbest_auc: {1:.4f}\tcolumn: {2}".format(h[0], best_auc, col))
    #print("selected columns: ", selected_cols)


pd.DataFrame(result, columns = ["column", "auc", "prev_auc",'gap']).to_csv("var_selection.csv")   


 ===========Test Uncertainty_3_dis============

# of obs: 53635
# of obs after drop na: 45611
# of positive obs: 407.0
iterate_num:  1 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.5828
 epoch:1 auc: 0.6133
 epoch:2 auc: 0.6303
 epoch:3 auc: 0.6348
 epoch:4 auc: 0.6341
 epoch:5 auc: 0.6365
 epoch:6 auc: 0.6363
 epoch:7 auc: 0.6365
 epoch:8 auc: 0.6400
 epoch:9 auc: 0.6414
AUC:  0.6414
iterate_num:  2 
 sum of test_y: 136.0
{0.0: 0.5044962835147332, 1.0: 56.101476014760145}
 epoch:0 auc: 0.5992
 epoch:1 auc: 0.6370
 epoch:2 auc: 0.6662
 epoch:3 auc: 0.6775
 epoch:4 auc: 0.6831
 epoch:5 auc: 0.6855
 epoch:6 auc: 0.6817
 epoch:7 auc: 0.6890
 epoch:8 auc: 0.6874
 epoch:9 auc: 0.6844
AUC:  0.6844
iterate_num:  3 
 sum of test_y: 135.0
{0.0: 0.5045128749668171, 1.0: 55.89705882352941}
 epoch:0 auc: 0.6093
 epoch:1 auc: 0.6338
 epoch:2 auc: 0.6518
 epoch:3 auc: 0.6669
 epoch:4 auc: 0.6696
 epoch:5 auc: 0.6787
 epoch:6 auc: 0.6806
 epoch:7 auc: 0.68

In [ ]:
print(random.shuffle(selected_new_all))